In [3]:
#import gzip
#import pandas as pd
#import numpy as np
#from sklearn.metrics import roc_auc_score, confusion_matrix, make_scorer, f1_score
#from sklearn.linear_model import LogisticRegression
#from sklearn.compose import make_column_selector as selector
#from sklearn.impute import SimpleImputer
#from sklearn.preprocessing import StandardScaler, OneHotEncoder
#from sklearn.compose import ColumnTransformer
#from sklearn.pipeline import Pipeline
#from sklearn.model_selection import train_test_split

# Settings for plots
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = [10, 7]
plt.rcParams['font.size'] = 15
sns.set(color_codes=True)
sns.set(font_scale=1.5)
sns.set_palette("bright")
sns.set_style("whitegrid")

#import automlx
#from automlx import init

In [4]:
# Fetch example data set
from sklearn.datasets import fetch_openml

dataset = fetch_openml(name='adult', as_frame=True)
df, y = dataset.data, dataset.target

HTTPError: HTTP Error 404: Not Found

In [ ]:
# Output header data for quick inspection
df.head()

In [ ]:
# Output default data types detected by the fetch (notice all are not correct, e.g., see "age" incorrectly listed as "category")
pd.DataFrame({'Data type': df.dtypes}).T

In [ ]:
# Output % missing values per column
pd.DataFrame({'% missing vlaues': df.isnull().sum() * 100 / len(df)}).T

In [ ]:
# visualize the distribution of the target variable (income)
y_df = pd.DataFrame(y)
y_df.columns = ['income']

ax = sns.countplot(x="income", data=y_df)
ax.bar_label(ax.containers[0])

In [ ]:
# Correct the incorrect column data types
numeric_columns = ['age', 'capitalgain', 'capitalloss', 'hoursperweek']
for col in df.columns:
    if col in numeric_columns:
        df[col] = df[col].astype(int)
        
        
# Split the data set into training and test data
X_train, X_test, y_train, y_test = train_test_split(df,
                                                    y.map({'>50K': 1, '<=50K': 0}).astype(int),
                                                    train_size=0.7,
                                                    random_state=0)

# Output the shap of the training and test data sets
X_train.shape, X_test.shape

In [ ]:
# Initialize the AutoML Engine, setting it to use Python's parallelism library
init(engine='local')

In [ ]:
# Create and fit a model using the AutoML Piepline (Preprocessing > Algorithm Selection > Adaptive Sampling > Feature Selection > Hyperparameter Tuning)
est1 = automlx.Pipeline(task='classification')
est1.fit(X_train, y_train)

In [ ]:
# Evaluate the effectiveness of the module
y_proba = est1.predict_proba(X_test)
score_default = roc_auc_score(y_test, y_proba[:, 1])

print(f'Score on test data: {score_default}')

In [ ]:
# Summarize the AutoML activity
est1.print_summary()

In [ ]:
# Graph the results from the "Model Selection" step/stage
trials_raw = est1.completed_trials_detailed_
trials = trials_raw[trials_raw['Step'] == 'Model Selection']

colors = []
scores = trials['Score (neg_log_loss)'].tolist()
models = trials['Algorithm'].tolist()

y_margin = 0.10 * (max(scores) - min(scores))
s = pd.Series(scores, index=models).sort_values(ascending=False)

for index, value in s.items():
    if index == est1.selected_model_:
        colors.append('orange')
    elif value >= s.mean():
        colors.append('teal')
    else:
        colors.append('turquoise')
        
fig, ax = plt.subplots(1)
ax.set_title("Algorithm Selection Trials")
ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
ax.set_ylabel('neg_log_loss')
s.plot.bar(ax=ax, color=colors, edgecolor='black')
ax.axhline(y=s.mean(), color='black', linewidth=0.5)
plt.show()

In [ ]:
# Graph the results from the "Adaptive Sampling" step/stage
trials = trials_raw[trials_raw['Step'] == 'Adaptive Sampling']
trials = trials[trials['Score (neg_log_loss)'] != -np.inf ]

scores    = trials['Score (neg_log_loss)'].tolist()
models    = trials['Algorithm'].tolist()
n_samples = trials['# Samples'].tolist()

y_margin = 0.10 * (max(scores) - min(scores))
        
fig, ax = plt.subplots(1)
ax.set_title("Adaptive Sampling ({})".format(models[0].removeprefix("AdaptiveSamplingStage_")))
ax.set_xlabel('Dataset sample size')
ax.set_ylabel('neg_log_loss')
ax.grid(color='g', linestyle='-', linewidth=0.1)
ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
ax.plot(n_samples, scores, 'k:', marker="s", color='teal', markersize=3)
plt.show()

In [ ]:
# Graph the results from the "Feature Selection" step/stage
print(f"Features selected: {est1.selected_features_names_}")
print(f"Features available: {df.keys().tolist()}")
dropped_features = df.keys().drop(est1.selected_features_names_, errors='ignore')
print(f"Features dropped: {dropped_features}") # <-- Need to take this output with a grain of salt as AutoMLX seemed to add "_#" indexes to ceratin feature names
 
trials = trials_raw[trials_raw['Step'] == 'Feature Selection']

scores    = trials['Score (neg_log_loss)'].tolist()
models    = trials['Algorithm'].tolist()
n_features = trials['# Features'].tolist()

y_margin = 0.10 * (max(scores) - min(scores))
        
fig, ax = plt.subplots(1)
ax.set_title("Feature Selection Trials")
ax.set_xlabel('Number of Features')
ax.set_ylabel('neg_log_loss')
ax.grid(color='g', linestyle='-', linewidth=0.1)
ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
ax.plot(n_features, scores, 'k:', marker="s", color='teal', markersize=3)
ax.axvline(x=len(est1.selected_features_names_), color='orange', linewidth=2.0)
plt.show()

In [ ]:
# Graphy the confusion matrix for our pipeline
y_pred = est1.predict(X_test)
cm = confusion_matrix(y_test.astype(int), y_pred, labels=[False, True])
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

df_cm = pd.DataFrame(cm)
df_cm = df_cm * 100

ax = sns.heatmap(df_cm, annot=True, fmt='.2f', cbar=False, xticklabels=['<=50K', '>50K'], yticklabels=['<=50K', '>50K'])
ax.set_xlabel('Prediction')
ax.set_ylabel('Actual')

In [ ]:
# Provide a specific model list to AutoML
est2 = automlx.Pipeline(model_list=['LogisticRegression']) # Default task type is "classification"
est2.fit(X_train, y_train)
y_proba = est2.predict_proba(X_test)
score_modellist = roc_auc_score(y_test, y_proba[:, 1])

print(f'ROC AUC Score on test data: {score_modellist}') # Expect many warning as 'LogisticRegression' is not a good match for this data (does not converge in most cases)


In [ ]:
# Use a custom validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.7, random_state=0)

est3 = automlx.Pipeline()
est3.fit(X_train, y_train, X_val, y_val)
y_proba = est3.predict_proba(X_test)
score_modellist = roc_auc_score(y_test, y_proba[:, 1])

print(f'ROC AUC Score on test data: {score_modellist}')

In [ ]:
# Pipeline Variation - Increase the number of tuned models
est4 = automlx.Pipeline(n_algos_tuned=2, model_list=['GaussianNB', 'LogisticRegression', 'XGBClassifier'])
est4.fit(X_train, y_train)

y_proba = est4.predict_proba(X_test)
score_modellist = roc_auc_score(y_test, y_proba[:, 1])

print(f'ROC AUC Score on test data: {score_modellist}')

In [ ]:
# Pipeline Variation - Specify a different scoring metric to AutoML
est5 = automlx.Pipeline(score_metric='accuracy', model_list=['XGBClassifier'])
est5.fit(X_train, y_train)

y_proba = est5.predict_proba(X_test)
score_modellist = roc_auc_score(y_test, y_proba[:, 1])

print(f'ROC AUC Score on test data: {score_modellist}')

In [ ]:
# Pipeline Variation - Specify a user defined scoring function to AutoML
#score_fn = make_scorer(f1_score, greater_is_better=True, needs_proba=False, average='macro')
def custom_mae_scorer(y_true, y_pred):
    """
    Custom Mean Absolute Error scorer.
    AutoMLx expects a function that takes (y_true, y_pred) as input.
    """
    def __init__(self, name="custom_mae_scorer"):
        self.__name__ = name
        self.y_true = y_true
        self.y_pred = y_pred
    
    return -mean_absolute_error(y_true, y_pred) # Return negative for optimization (AutoMLx maximizes scores)

est6 = automlx.Pipeline(
    task='classification',
    score_metric=custom_mae_scorer, # Or custom_mae_scorer if not using make_scorer
    random_state=42
)
est6.fit(X_train, y_train)

y_proba = est6.predict_proba(X_test)
score_modellist = roc_auc_score(y_test, y_proba[:, 1])

print(f'ROC AUC Score on test data: {score_modellist}')

In [ ]:
# Pipeline Variation - Specify a time budget to AutoML
est7 = automlx.Pipeline()
est7.fit(X_train, y_train, time_budget=10) # <-- This made an amazing difference in auto-modeling time, but did not change the score substantially

y_proba = est7.predict_proba(X_test)
score_modellist = roc_auc_score(y_test, y_proba[:, 1])

print(f'ROC AUC Score on test data: {score_modellist}')

In [ ]:
# Pipeline Variation - Specify a time budget to AutoML
est8 = automlx.Pipeline(min_features=['fnlwgt', 'native-country'], model_list=['XGBClassifier'])
est8.fit(X_train, y_train)

y_proba = est8.predict_proba(X_test)
score_modellist = roc_auc_score(y_test, y_proba[:, 1])

print(f'ROC AUC Score on test data: {score_modellist}')
print(f'Selected features: {est8.selected_features_names_}')